In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")
from IPython.display import HTML
# from matplotlib import rcParams
from matplotlib import animation
#rcParams['font.family']='sans-serif' 
#rcParams('font', serif='Helvetica Neue') 
# rcParams['text.usetex']= True 
#rcParams.update({'font.size': 22})
%matplotlib inline
#%matplotlib nbagg

In [ ]:
# Position verlet
class TimeStepper(object):
    """
    """
    def __init__(self, i_x, i_v, i_dt, i_T):
        """ Initialize
        """
        self.forcing = None
        self.timestepper = None
        self.nsteps = int(i_T/i_dt)
        self.dt = i_dt

        if len(i_x) == len(i_v):
            # Same length, corresponding to index
            self.ndim = len(i_x)
            self.x = np.zeros((self.nsteps, self.ndim))
            self.v = np.zeros((self.nsteps, self.ndim))
            
            # Set initial values
            self.x[0] = np.array(i_x)
            self.v[0] = np.array(i_v)
        else:
            raise RuntimeError('Length of initial velocity and position \
            not matching')

    def set_forcing_function(self, t_func):
        """ Set forcing function to be used 
        """

        if type(t_func) is not str:
            try:
                t_func(x[0])
            except:
                raise RuntimeError('Provided function cannot be evaluated')
        else:
            if t_func=="harmonic":
                def harmonic(x):
                    return -x
                self.forcing = harmonic
        
    def timestep_using(self, timestepper):
        """ Access to internal variables
        """
        self.timestepper = timestepper.__name__
        for i in range(self.nsteps - 1):
            # Do one cycle
            self.x[i+1], self.v[i+1] = timestepper(self.dt, self.x[i], self.v[i], self.forcing)
    
    def draw(self, renderer):
        if self.timestepper:
            renderer.scatter(self.x[0], self.v[0], c='k',marker='o')
            renderer.plot(self.x, self.v, label=self.timestepper)
            x_min, x_max = np.min(self.x), np.max(self.x)
            v_min, v_max = np.min(self.v), np.max(self.v)
            extension = 0.5
            renderer.set_xlim(min(0.0, x_min) - extension, max(0.0, x_max) + extension)
            renderer.set_ylim(min(0.0, x_min) - extension, max(0.0, v_max) + extension)
            renderer.legend()
        else:
            if self.forcing.__name__ == 'harmonic':
                true_sol = plt.Circle((0, 0), 1.0, fill=None, edgecolor='k', linestyle='--', lw=4)
                renderer.set_xlim(-1.5, 1.5)
                renderer.set_ylim(-1.5, 1.5)
                renderer.add_artist(true_sol)
            # raise RuntimeError('No information found. Did you forget to run your timestepper?')
        renderer.set_xlabel(r'$x$')
        renderer.set_ylabel(r'$v$')
        renderer.set_title(r'${}$'.format(self.forcing.__name__))
        renderer.set_aspect('equal')

    def animate(self, fig, renderer, color):
        self.timestepper = None
        # animation function. This is called sequentially
        def animate_in(i):
            renderer.clear()
            self.draw(renderer)
            for j in range(i + 1):
                renderer.plot([self.x[j], self.x[j+1]], [self.v[j], self.v[j+1]], marker='o', c=color, alpha=0.5**((i-j)/20.))
        # ax2.plot([b.x[i], b.x[i+1]], [b.v[i], b.v[i+1]], marker='o', c='g')
        #      line.set_data(b.x[i], b.v[i])
        #      return (line,)

        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, animate_in, frames=100, interval=5)

        return anim

In [ ]:
i_x = [1.0]
i_v = [0.0]
dt = 1.0
f_T = 628.0

fig, ax = plt.subplots(1,1, figsize=(10, 10))
a = TimeStepper(i_x, i_v, dt, f_T)
a.set_forcing_function('harmonic')
a.draw(ax)
fig.savefig('true_solution.pdf')

In [ ]:
def position_verlet(dt, x, v, force_rule):
    temp_x = x + 0.5*dt*v
    v_n = v + dt * force_rule(temp_x)
    x_n = temp_x + 0.5 * dt * v_n
    return x_n, v_n

a.timestep_using(position_verlet)

In [ ]:
a.draw(ax)
fig
# fig.savefig('position_verlet_1.0.pdf')

In [ ]:
def velocity_verlet(dt, x, v, force_rule):
    temp_v = v + 0.5*dt*force_rule(x)
    x_n = x + dt * temp_v
    v_n = temp_v + 0.5* dt * force_rule(x_n)
    return x_n, v_n

# a.timestep_using(velocity_verlet)
# a.draw(ax)
# fig

In [ ]:
def euler_fwd(dt, x, v, force_rule):
    x_n = x + dt * v
    v_n = v + dt * force_rule(x)
    return x_n, v_n

a.timestep_using(euler_fwd)
a.draw(ax)
fig
# fig.savefig('euler_fwd_1.0.pdf')

In [ ]:
def euler_cromer(dt, x, v, force_rule):
    x_n = x + dt * v
    v_n = v + dt * force_rule(x_n)
    return x_n, v_n

# a.timestep_using(euler_cromer)
# a.draw(ax)
# fig

In [ ]:
def runge_kutta4(dt, x, v, force_rule):

    def vector_func(y):
        return np.array([y[1], force_rule(y[0])])

    # Base
    u = np.array([x,v])

    # Stage 1
    k_1 = dt*vector_func(u)

    # Stage 2
    k_2 = dt * vector_func(u + 0.5*k_1)

    # Stage 3
    k_3 = dt * vector_func(u + 0.5*k_2)

    # Stage 4
    k_4 = dt * vector_func(u + k_3)

    u_n = u + (1./6.)*(k_1 + 2.*k_2 + 2.* k_3 + k_4)
    
    return u_n[0], u_n[1]

a.timestep_using(runge_kutta4)
a.draw(ax)
fig
# fig.savefig('rk4_1.0.pdf')

In [ ]:
fig2, ax2 = plt.subplots(1,1, figsize=(10, 10))
i_x = [0.0]
i_v = [1.0]
dt = 1.0
f_T = 700.0
b = TimeStepper(i_x, i_v, dt, f_T)
b.set_forcing_function('harmonic')
b.draw(ax2)
b.timestep_using(position_verlet)
ax2.set_xlim([-1.5, 1.5])
ax2.set_ylim([-1.5, 1.5])
#b.draw(ax2)
# fig2

In [ ]:
cmap = sns.color_palette()
verlet_color = cmap[0]
rk_color = cmap[2]
anim = b.animate(fig2, ax2, verlet_color)

In [ ]:
# HTML(anim.to_jshtml())

In [ ]:
anim.save('verlet.mp4', fps=30, 
          extra_args=['-vcodec', 'h264', 
                      '-pix_fmt', 'yuv420p'])

In [ ]:
b.nsteps